In [ ]:
import os
import json
import pickle

from utils.data_dirs import data_dirs
from utils.get_location import get_location
from utils.index import index

In [ ]:
EXPERIMENT = False

In [ ]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
with open(os.path.join(raw_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(raw_dir, folder, 'log.json')) as f:
    log_json = json.load(f)

with open(os.path.join(processed_dir, 'sample_data.pickle'), "rb") as f:
    sample_data = pickle.load(f)

with open(os.path.join(processed_dir, 'annotation.pickle'), "rb") as f:
    annotation = pickle.load(f)

In [ ]:
sample_data

In [ ]:
annotation

In [ ]:
scene_json[0]

In [ ]:
log_json[0]

In [ ]:
scene_map = index(scene_json)
log_map = index(log_json)

In [ ]:
data = {
    'scene': scene_map,
    'log': log_map
}

In [ ]:
locations = {}
for s in scene_json:
    location = log_map[s['log_token']]['location']
    
    if location not in locations:
        locations[location] = []
    locations[location].append(s)

In [ ]:
annotation[:1]

In [ ]:
a = annotation.reset_index()
for l, s in locations.items():
    scenes = {_s['name'] for _s in s}
    _sample_data = sample_data[sample_data['scene_name'].isin(scenes)]
    sample_tokens = {*_sample_data['sample_token'].tolist()}
    _annotation = a[a['sample_token'].isin(sample_tokens)]
    base = os.path.join(processed_dir, 'partition', l)
    if not os.path.exists(base):
        os.makedirs(base)
    with open(os.path.join(base, 'sample_data.pickle'), "wb") as f:
        pickle.dump(_sample_data, f)
    with open(os.path.join(base, 'annotation.pickle'), "wb") as f:
        pickle.dump(_annotation, f)
